In [ ]:
import torch

file_path = '/home/mzi/olp-gnn/dataset_train.pt'
data = torch.load(file_path)

new_data = []
k = 8
for nUes in [2, 4, 6]:
    for connProb in [1.0]:
        key = f'seed_100_precoder_olp_mor_urban_nAps_20_nUes_{nUes}_connProb_{connProb}'
        for data_ in data[0][key]:
            data_2 = data_.clone()
            channel_abs = data_2['channel'].x[:,0].reshape((nUes, 20))
            indices = torch.topk(channel_abs, k, dim=1, largest=True).indices
            mask = torch.zeros((nUes, 20)).scatter_(1, indices, 1).to(torch.int)
            data_2['channel'].y = mask.reshape((-1, 1))
            data_2['channel'].x = data_2['channel'].x[:,:2].reshape((-1, 2))
            data_2['channel'].input_mean = data_2['channel'].input_mean[:,:2]
            data_2['channel'].input_std = data_2['channel'].input_std[:,:2]
            data_2['channel'].pop("output_mean")
            data_2['channel'].pop("output_std")
            data_2['channel'].pop("rho_d")
            data_2['channel'].pop("sinr")
            new_data.append(data_2)
            


/tmp/ipykernel_2983075/884244392.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  data = torch.load(file_path)


In [114]:
new_data[110]

HeteroData(
  channel={
    x=[40, 2],
    y=[40, 1],
    input_mean=[1, 2],
    input_std=[1, 2],
    n_ues=2,
    n_aps=20,
    num_graph_node=40,
  },
  (channel, same_ue, channel)={ edge_index=[2, 760] },
  (channel, same_ap, channel)={ edge_index=[2, 40] }
)

In [98]:
import pickle
with open("/home/mzi/olp-gnn/data_generation/8strongest/data.pickle", 'wb') as f:
    pickle.dump(new_data, f)